# sKCSD tutorial
In this tutorial we will cover three topics: data format for sKCSD estimation, sKCSD method and source visualization. 

In [5]:
from kcsd import sKCSDcell, sKCSD, sample_data_path
from kcsd import utility_functions as utils
from kcsd.utility_functions import LoadData
import os

In [7]:
data_fname = os.path.join(sample_data_path, "gang_7x7_200")


In [8]:
data = LoadData(fname)

Load /home/jszmek/kCSD-python/kcsd/../data/gang_7x7_200/morphology/Badea2011Fig2Du.CNG.swc
Load /home/jszmek/kCSD-python/kcsd/../data/gang_7x7_200/electrode_positions/elcoord_x_y_z
Load /home/jszmek/kCSD-python/kcsd/../data/gang_7x7_200/LFP/myLFP


Data should be divided into three subdirectories: morphology, electrode_positions and LFP, each containing one file with morphology, electrode_positions and LFP. LoadData currently supports only swc morphology format. LoadData can read in electrode positions as a text file either with 1 column with x postions for each electrode followed by y postions for each electrodes followed by z positions of each electrode; or a textfile with 3 columns with x, y, z electrode postions.  LFPs should be a text file with appropriate numbers of the shape of n_electrodes x n_time_samples. LoadData also allows for initialization of an empty object and reading in arbitrary data files from specific location using assign function: 
data1 = LoadData()
data1.assign('morphology', path_to_morphology_file)